In [1]:
#Importing Libraries
import numpy as np
import pandas as pd
import pickle
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from functools import reduce
from pyspark.sql.types import ArrayType, StringType
import re

In [2]:
from IPython.core.display import HTML,display
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [3]:
jdf = [
  {
    "upi_transaction_id": "YBL3b55ac093d76488c9cef1425265396cd",
    "transaction_type": "PAY",
    "transaction_reference_id": "1200233635",
    "customer_reference_number": 203230251039,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 75000,
    "payee_account_number": "918020110872063",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "UTIB0000000",
    "payee_name": "PAVAN BAR",
    "payee_vpa": "gpay-11200233635@okbizaxis",
    "payee_amount": 75000,
    "state": "SUCCESS",
    "created": "2022-02-01T19:58:44.300Z",
    "updated": "2022-02-01T19:58:46.885Z",
    "data_dt": "2022-02-07T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL2a3a489ee99e44caa56910bc3cc110d1",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2201061024416257077813",
    "customer_reference_number": 200680911557,
    "note": "Payment from PhonePe",
    "payer_account_number": 95425001007111540,
    "payer_account_type": "CURRENT",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 72000,
    "payee_account_number": 9542500100516700,
    "payee_account_type": "CURRENT",
    "payee_ifsc": "ANDB0001491",
    "payee_name": "SHANKARLINGAPPA I HUGAR",
    "payee_vpa": "9008819966@ybl",
    "payee_amount": 72000,
    "state": "SUCCESS",
    "created": "2022-01-06T10:24:45.075Z",
    "updated": "2022-01-06T10:24:47.867Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number":95425001007111540,
    "account_type": "CURRENT",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL2a3a489ee99e44caa56910bc3cc110d1",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2201061024416257077813",
    "customer_reference_number": 200680911557,
    "note": "Payment from PhonePe",
    "payer_account_number": 95425001007111540,
    "payer_account_type": "CURRENT",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 72000,
    "payee_account_number": 9542500100516700,
    "payee_account_type": "CURRENT",
    "payee_ifsc": "ANDB0001491",
    "payee_name": "SHANKARLINGAPPA I HUGAR",
    "payee_vpa": "9008819966@ybl",
    "payee_amount": 72000,
    "state": "SUCCESS",
    "created": "2022-01-06T10:24:45.075Z",
    "updated": "2022-01-06T10:24:47.867Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 95425001007111540,
    "account_type": "CURRENT",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL2a3a489ee99e44caa56910bc3cc110d1",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2201061024416257077813",
    "customer_reference_number": 200680911557,
    "note": "Payment from PhonePe",
    "payer_account_number": 95425001007111540,
    "payer_account_type": "CURRENT",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 72000,
    "payee_account_number": 9542500100711100,
    "payee_account_type": "CURRENT",
    "payee_ifsc": "ANDB0001491",
    "payee_name": "SHANKARLINGAPPA I HUGAR",
    "payee_vpa": "9008819966@ybl",
    "payee_amount": 72000,
    "state": "SUCCESS",
    "created": "2022-01-06T10:24:45.075Z",
    "updated": "2022-01-06T10:24:47.867Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 95425001007111540,
    "account_type": "CURRENT",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL2a3a489ee99e44caa56910bc3cc110d1",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2201061024416257077813",
    "customer_reference_number": 200680911557,
    "note": "Payment from PhonePe",
    "payer_account_number": 95425001007111540,
    "payer_account_type": "CURRENT",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 72000,
    "payee_account_number": 9542500100711100,
    "payee_account_type": "CURRENT",
    "payee_ifsc": "ANDB0001491",
    "payee_name": "SHANKARLINGAPPA I HUGAR",
    "payee_vpa": "9008819966@ybl",
    "payee_amount": 72000,
    "state": "SUCCESS",
    "created": "2022-01-06T10:24:45.075Z",
    "updated": "2022-01-06T10:24:47.867Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 95425001007111540,
    "account_type": "CURRENT",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL2a3a489ee99e44caa56910bc3cc110d1",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2201061024416257077813",
    "customer_reference_number": 200680911557,
    "note": "Payment from PhonePe",
    "payer_account_number": 95425001007111540,
    "payer_account_type": "CURRENT",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 72000,
    "payee_account_number": "149110100033175",
    "payee_account_type": "CURRENT",
    "payee_ifsc": "ANDB0001491",
    "payee_name": "SHANKARLINGAPPA I HUGAR",
    "payee_vpa": "9008819966@ybl",
    "payee_amount": 72000,
    "state": "SUCCESS",
    "created": "2022-01-06T10:24:45.075Z",
    "updated": "2022-01-06T10:24:47.867Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 95425001007111540,
    "account_type": "CURRENT",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL2a3a489ee99e44caa56910bc3cc110d1",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2201061024416257077813",
    "customer_reference_number": 200680911557,
    "note": "Payment from PhonePe",
    "payer_account_number": 95425001007111540,
    "payer_account_type": "CURRENT",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 72000,
    "payee_account_number": "149110100033175",
    "payee_account_type": "CURRENT",
    "payee_ifsc": "ANDB0001491",
    "payee_name": "SHANKARLINGAPPA I HUGAR",
    "payee_vpa": "9008819966@ybl",
    "payee_amount": 72000,
    "state": "SUCCESS",
    "created": "2022-01-06T10:24:45.075Z",
    "updated": "2022-01-06T10:24:47.867Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 95425001007111540,
    "account_type": "CURRENT",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL2a3a489ee99e44caa56910bc3cc110d1",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2201061024416257077813",
    "customer_reference_number": 200680911557,
    "note": "Payment from PhonePe",
    "payer_account_number": 95425001007111540,
    "payer_account_type": "CURRENT",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 72000,
    "payee_account_number": "149110100033175",
    "payee_account_type": "CURRENT",
    "payee_ifsc": "ANDB0001491",
    "payee_name": "SHANKARLINGAPPA I HUGAR",
    "payee_vpa": "9008819966@ybl",
    "payee_amount": 72000,
    "state": "SUCCESS",
    "created": "2022-01-06T10:24:45.075Z",
    "updated": "2022-01-06T10:24:47.867Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 95425001007111540,
    "account_type": "CURRENT",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL2a3a489ee99e44caa56910bc3cc110d1",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2201061024416257077813",
    "customer_reference_number": 200680911557,
    "note": "Payment from PhonePe",
    "payer_account_number": 95425001007111540,
    "payer_account_type": "CURRENT",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 72000,
    "payee_account_number": "149110100033175",
    "payee_account_type": "CURRENT",
    "payee_ifsc": "ANDB0001491",
    "payee_name": "SHANKARLINGAPPA I HUGAR",
    "payee_vpa": "9008819966@ybl",
    "payee_amount": 72000,
    "state": "SUCCESS",
    "created": "2022-01-06T10:24:45.075Z",
    "updated": "2022-01-06T10:24:47.867Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 95425001007111540,
    "account_type": "CURRENT",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL2a3a489ee99e44caa56910bc3cc110d1",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2201061024416257077813",
    "customer_reference_number": 200680911557,
    "note": "Payment from PhonePe",
    "payer_account_number": 95425001007111540,
    "payer_account_type": "CURRENT",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 72000,
    "payee_account_number": "149110100033175",
    "payee_account_type": "CURRENT",
    "payee_ifsc": "ANDB0001491",
    "payee_name": "SHANKARLINGAPPA I HUGAR",
    "payee_vpa": "9008819966@ybl",
    "payee_amount": 72000,
    "state": "SUCCESS",
    "created": "2022-01-06T10:24:45.075Z",
    "updated": "2022-01-06T10:24:47.867Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 95425001007111540,
    "account_type": "CURRENT",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL2a3a489ee99e44caa56910bc3cc110d1",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2201061024416257077813",
    "customer_reference_number": 200680911557,
    "note": "Payment from PhonePe",
    "payer_account_number": 95425001007111540,
    "payer_account_type": "CURRENT",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 72000,
    "payee_account_number": "149110100033175",
    "payee_account_type": "CURRENT",
    "payee_ifsc": "ANDB0001491",
    "payee_name": "SHANKARLINGAPPA I HUGAR",
    "payee_vpa": "9008819966@ybl",
    "payee_amount": 72000,
    "state": "SUCCESS",
    "created": "2022-01-06T10:24:45.075Z",
    "updated": "2022-01-06T10:24:47.867Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 95425001007111540,
    "account_type": "CURRENT",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBLdc58fbba10be465e8991493bf6632ed8",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2202011229232764503394",
    "customer_reference_number": 203272206073,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 1800,
    "payee_account_number": "00000054007513786",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "SBIN0040605",
    "payee_name": "SANJEEVA  C",
    "payee_vpa": "Q513055970@ybl",
    "payee_amount": 1800,
    "state": "SUCCESS",
    "created": "2022-02-01T12:29:25.697Z",
    "updated": "2022-02-01T12:29:29.521Z",
    "data_dt": "2022-02-07T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBLdbc43d56f6764418857167defc91ef16",
    "transaction_type": "PAY",
    "transaction_reference_id": "1200025300",
    "customer_reference_number": 200777647233,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 9000,
    "payee_account_number": "918020110872063",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "UTIB0000000",
    "payee_name": "G N STORES",
    "payee_vpa": "gpay-11200025300@okbizaxis",
    "payee_amount": 9000,
    "state": "SUCCESS",
    "created": "2022-01-07T12:18:23.183Z",
    "updated": "2022-01-07T12:18:28.495Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL6f7be574f7a84d37ac6f32f040ffabb4",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2202030848314044720230",
    "customer_reference_number": 203481181335,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 1800,
    "payee_account_number": "00000054007513786",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "SBIN0040605",
    "payee_name": "SANJEEVA  C",
    "payee_vpa": "Q513055970@ybl",
    "payee_amount": 1800,
    "state": "SUCCESS",
    "created": "2022-02-03T08:48:35.539Z",
    "updated": "2022-02-03T08:48:37.328Z",
    "data_dt": "2022-02-07T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL36518a861bf440cba731b9929db8eabf",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2202032215493445088110",
    "customer_reference_number": 203423441798,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 200000,
    "payee_account_number": "5445715243",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "KKBK0008062",
    "payee_name": "ASHOK   K",
    "payee_vpa": "9986528460@ybl",
    "payee_amount": 200000,
    "state": "SUCCESS",
    "created": "2022-02-03T22:15:52.653Z",
    "updated": "2022-02-03T22:15:54.833Z",
    "data_dt": "2022-02-07T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL98cd64615a3842939a0dd6f6277b75a6",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2202252126498206823913",
    "customer_reference_number": 205647631673,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 40000,
    "payee_account_number": "15042600001897",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "CNRB0000033",
    "payee_name": "DARSHAN D V",
    "payee_vpa": "darshan.dv@ybl",
    "payee_amount": 40000,
    "state": "SUCCESS",
    "created": "2022-02-25T21:26:52.638Z",
    "updated": "2022-02-25T21:26:54.200Z",
    "data_dt": "2022-02-27T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBLbf83393e82e4454f991b4e79c5445655",
    "transaction_type": "PAY",
    "transaction_reference_id": "1197405594",
    "customer_reference_number": 203134457255,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 1800,
    "payee_account_number": "918020110872063",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "UTIB0000000",
    "payee_name": "Paan Stall",
    "payee_vpa": "gpay-11197405594@okbizaxis",
    "payee_amount": 1800,
    "state": "SUCCESS",
    "created": "2022-01-31T14:06:25.823Z",
    "updated": "2022-01-31T14:06:28.077Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL0ccf09ef9a52416cb4135bc4e9fc4b83",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2202060935527432168012",
    "customer_reference_number": 203728933198,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 4700,
    "payee_account_number": "00000054007513786",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "SBIN0040605",
    "payee_name": "SANJEEVA  C",
    "payee_vpa": "Q513055970@ybl",
    "payee_amount": 4700,
    "state": "SUCCESS",
    "created": "2022-02-06T09:35:55.546Z",
    "updated": "2022-02-06T09:35:57.076Z",
    "data_dt": "2022-02-07T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL7aad4fb33d9e4345a4d8dc9becf945d3",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2202071612588899207083",
    "customer_reference_number": 203839653851,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 5000,
    "payee_account_number": "4219101002468",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "CNRB0000000",
    "payee_name": "VIJAY C M",
    "payee_vpa": "8197534210@ybl",
    "payee_amount": 5000,
    "state": "SUCCESS",
    "created": "2022-02-07T16:13:01.681Z",
    "updated": "2022-02-07T16:13:04.356Z",
    "data_dt": "2022-02-07T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL558f8ca626634ced9e51babad7f133ea",
    "transaction_type": "PAY",
    "transaction_reference_id": "1173717007",
    "customer_reference_number": 204815288220,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 1500,
    "payee_account_number": "918020110872063",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "UTIB0000000",
    "payee_name": "A1 Plantain Mandy",
    "payee_vpa": "gpay-11173717007@okbizaxis",
    "payee_amount": 1500,
    "state": "SUCCESS",
    "created": "2022-02-17T20:51:01.473Z",
    "updated": "2022-02-17T20:51:03.110Z",
    "data_dt": "2022-02-19T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBLa69c2955ef8c4a68a2b8745db7191dd1",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2201041657290521255407",
    "customer_reference_number": 200447386801,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100711100,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 1100000,
    "payee_account_number": "520191054361520",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "CORP0000098",
    "payee_name": "JOVESTER DSOUZA",
    "payee_vpa": "9964507166@ybl",
    "payee_amount": 1100000,
    "state": "SUCCESS",
    "created": "2022-01-04T16:57:33.524Z",
    "updated": "2022-01-04T16:57:37.698Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 9542500100711100,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL9da71b83cde64a12a7ded6521fe15470",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2202101904307438127077",
    "customer_reference_number": 204111617373,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100711100,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 26000,
    "payee_account_number": 95425001007111540,
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "LAVB0000161",
    "payee_name": "KALU RAM",
    "payee_vpa": "Q93659325@ybl",
    "payee_amount": 26000,
    "state": "SUCCESS",
    "created": "2022-02-10T19:04:35.591Z",
    "updated": "2022-02-10T19:04:38.689Z",
    "data_dt": "2022-02-12T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 9542500100711100,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL2a3a489ee99e44caa56910bc3cc110d1",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2201061024416257077813",
    "customer_reference_number": 200680911557,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100711100,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 72000,
    "payee_account_number": 95425001007111540,
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "ANDB0001491",
    "payee_name": "SHANKARLINGAPPA I HUGAR",
    "payee_vpa": "9008819966@ybl",
    "payee_amount": 72000,
    "state": "SUCCESS",
    "created": "2022-01-06T10:24:45.075Z",
    "updated": "2022-01-06T10:24:47.867Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 9542500100711100,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL2a3a489ee99e44caa56910bc3cc110d1",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2201061024416257077813",
    "customer_reference_number": 200680911557,
    "note": "Payment from PhonePe",
    "payer_account_number": 95425001007111540,
    "payer_account_type": "CURRENT",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 72000,
    "payee_account_number": "149110100033175",
    "payee_account_type": "CURRENT",
    "payee_ifsc": "ANDB0001491",
    "payee_name": "SHANKARLINGAPPA I HUGAR",
    "payee_vpa": "9008819966@ybl",
    "payee_amount": 72000,
    "state": "SUCCESS",
    "created": "2022-01-06T10:24:45.075Z",
    "updated": "2022-01-06T10:24:47.867Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 95425001007111540,
    "account_type": "CURRENT",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "APLe2773aa50c1847a7b2b7233aa9223da6",
    "transaction_type": "COLLECT",
    "transaction_reference_id": "TQhuK1uH4fZUdaE6GhNFsVsxka2j1mrRasG",
    "customer_reference_number": 201115582828,
    "note": "Quant MF",
    "payer_account_number": 9542500100711100,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 104700,
    "payee_account_number": 95425001007111540,
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "UTIB0000100",
    "payee_name": "Quant Mutual Fund",
    "payee_vpa": "9445292214@ybl",
    "payee_amount": 104700,
    "state": "SUCCESS",
    "created": "2022-01-11T15:01:13.297Z",
    "updated": "2022-01-11T15:01:34.397Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 9542500100711100,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Investment",
    "Category2": "Mutualfund_transactions"
  },
  {
    "upi_transaction_id": "APLe2773aa50c1847a7b2b7233aa9223da6",
    "transaction_type": "COLLECT",
    "transaction_reference_id": "TQhuK1uH4fZUdaE6GhNFsVsxka2j1mrRasG",
    "customer_reference_number": 201115582828,
    "note": "Quant MF",
    "payer_account_number": 9542500100711100,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 104700,
    "payee_account_number": 95425001007111540,
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "UTIB0000100",
    "payee_name": "Quant Mutual Fund",
    "payee_vpa": "9445292214@ybl",
    "payee_amount": 104700,
    "state": "SUCCESS",
    "created": "2022-01-11T15:01:13.297Z",
    "updated": "2022-01-11T15:01:34.397Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 9542500100711100,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Investment",
    "Category2": "Mutualfund_transactions"
  }]

In [4]:
df = spark.read.json(sc.parallelize([jdf]))
df.show(truncate=False)

+-----------------+---------------------------+-------------------+------------+-----------------+------------+------------------------+-------------------------+------------------------+----------------+--------------------+--------------------+------------------+------------+-----------+-----------------------+--------------------------+--------------------+------------------+------------+-----------+----------+--------------+------------+------------------------+-------+------------------------+----------------+------------------------+-----------------------------------+
|Category1        |Category2                  |account_holder_name|account_ifsc|account_number   |account_type|created                 |customer_reference_number|data_dt                 |mbanking_enabled|note                |payee_account_number|payee_account_type|payee_amount|payee_ifsc |payee_name             |payee_vpa                 |payer_account_number|payer_account_type|payer_amount|payer_ifsc |payer_name|p

In [5]:
cols = ("Category1","Category2")
df2 = df.drop(*cols)

In [6]:
df2.show(truncate=False)

+-------------------+------------+-----------------+------------+------------------------+-------------------------+------------------------+----------------+--------------------+--------------------+------------------+------------+-----------+-----------------------+--------------------------+--------------------+------------------+------------+-----------+----------+--------------+------------+------------------------+-------+------------------------+----------------+------------------------+-----------------------------------+
|account_holder_name|account_ifsc|account_number   |account_type|created                 |customer_reference_number|data_dt                 |mbanking_enabled|note                |payee_account_number|payee_account_type|payee_amount|payee_ifsc |payee_name             |payee_vpa                 |payer_account_number|payer_account_type|payer_amount|payer_ifsc |payer_name|payer_vpa     |phone_number|registered_at_acct      |state  |transaction_reference_id|transac

In [7]:
# Define the custom tokenizer
def custom_tokenizer(text):
    pattern = re.compile(r'[a-zA-Z]+\d+')
    return pattern.findall(text)

# Load the saved models and vectorizers
clf_cat1 = pickle.load(open("clf_cat1.pkl", "rb"))
clf_cat2 = pickle.load(open("clf_cat2.pkl", "rb"))
tfidf_payer_name = pickle.load(open("tfidf_payer_name.pkl", "rb"))
tfidf_payee_name = pickle.load(open("tfidf_payee_name.pkl", "rb"))
tfidf_payee_account_type = pickle.load(open("tfidf_payee_account_type.pkl", "rb"))
tfidf_payer_account_type = pickle.load(open("tfidf_payer_account_type.pkl", "rb"))
tfidf_payer_vpa = pickle.load(open("tfidf_payer_vpa.pkl", "rb"))
tfidf_payee_vpa = pickle.load(open("tfidf_payee_vpa.pkl", "rb"))


# Define the prediction function
def predict_categories(payer_name, payee_name, payee_account_type,
                       payer_account_type, payer_vpa, payee_vpa):
    # Transform input data using the TFIDF vectorizers
    payer_name_vec = tfidf_payer_name.transform([payer_name])
    payee_name_vec = tfidf_payee_name.transform([payee_name])
    payee_account_type_vec = tfidf_payee_account_type.transform([payee_account_type])
    payer_account_type_vec = tfidf_payer_account_type.transform([payer_account_type])
    payer_vpa_vec = tfidf_payer_vpa.transform([payer_vpa])
    payee_vpa_vec = tfidf_payee_vpa.transform([payee_vpa])

    tfidf_matrix = pd.concat([pd.DataFrame(payer_name_vec.toarray()),
                              pd.DataFrame(payee_name_vec.toarray()),
                              pd.DataFrame(payee_account_type_vec.toarray()),
                              pd.DataFrame(payer_account_type_vec.toarray()),
                              pd.DataFrame(payer_vpa_vec.toarray()),
                              pd.DataFrame(payee_vpa_vec.toarray())], axis=1)

    # Predict
    prediction_cat1 = clf_cat1.predict(tfidf_matrix)
    prediction_cat2 = clf_cat2.predict(tfidf_matrix)

    return [prediction_cat1[0], prediction_cat2[0]]

# Register the prediction function as a UDF
predict_udf = udf(predict_categories, ArrayType(StringType()))

# Make sure your DataFrame has columns: payer_name, payee_name, payee_account_type, payer_account_type, payer_vpa, payee_vpa
result_df = df2.withColumn("predictions", predict_udf("payer_name", "payee_name", "payee_account_type", "payer_account_type", "payer_vpa", "payee_vpa"))
# Assuming result_df is the DataFrame with the combined predictions column
result_df = result_df.withColumn("category_level1", result_df["predictions"].getItem(0))
result_df = result_df.withColumn("category_level2", result_df["predictions"].getItem(1))

# Dropping the combined predictions column
result_df = result_df.drop("predictions")

# Show the resulting DataFrame
result_df.show()

+-------------------+------------+-----------------+------------+--------------------+-------------------------+--------------------+----------------+--------------------+--------------------+------------------+------------+-----------+--------------------+--------------------+--------------------+------------------+------------+-----------+----------+--------------+------------+--------------------+-------+------------------------+----------------+--------------------+--------------------+-----------------+--------------------+
|account_holder_name|account_ifsc|   account_number|account_type|             created|customer_reference_number|             data_dt|mbanking_enabled|                note|payee_account_number|payee_account_type|payee_amount| payee_ifsc|          payee_name|           payee_vpa|payer_account_number|payer_account_type|payer_amount| payer_ifsc|payer_name|     payer_vpa|phone_number|  registered_at_acct|  state|transaction_reference_id|transaction_type|            

In [8]:
result_df.select("category_level1","category_level2").distinct().show()

+-----------------+--------------------+
|  category_level1|     category_level2|
+-----------------+--------------------+
|       Investment|Mutualfund_transa...|
|         Shopping|             offline|
|Personal Transfer|TRANSFER_FRIENDS_...|
+-----------------+--------------------+



In [11]:
from pyspark.sql import functions as F

def compute_aggregates(df, category_col, amount_col):
    # Pivot on the category column to compute counts and sums for each category
    agg_df = df.groupBy("payer_account_number", "payer_account_type").pivot(category_col).agg(
        F.count(amount_col).alias("count"),
        F.sum(amount_col).alias("sum")
    )
    
    # Rename columns and add value type column based on the sum
    for category in df.select(category_col).distinct().rdd.flatMap(lambda x: x).collect():
        agg_df = agg_df.withColumnRenamed(category + "_count", "count_" + category) \
                      .withColumnRenamed(category + "_sum", "sum_" + category) \
                      .withColumn("type_" + category, 
                                  F.when((F.col("count_" + category) == 0) | (F.col("sum_" + category).isNull()) | (F.col("sum_" + category) == 0), "No transactions")
                                  .when(F.col("payer_account_type") == "SAVINGS", 
                                        F.when(F.col("sum_" + category) < 5000000, "10")
                                        .when((F.col("sum_" + category) >= 5000000) & (F.col("sum_" + category) < 10000000), "9")
                                        .when((F.col("sum_" + category) >= 10000000) & (F.col("sum_" + category) < 15000000), "8")
                                        .when((F.col("sum_" + category) >= 15000000) & (F.col("sum_" + category) < 20000000), "7")
                                        .when((F.col("sum_" + category) >= 20000000) & (F.col("sum_" + category) < 25000000), "6")
                                        .when((F.col("sum_" + category) >= 25000000) & (F.col("sum_" + category) < 30000000), "5")
                                        .when((F.col("sum_" + category) >= 30000000) & (F.col("sum_" + category) < 35000000), "4")
                                        .when((F.col("sum_" + category) >= 35000000) & (F.col("sum_" + category) < 40000000), "3")
                                        .when((F.col("sum_" + category) >= 40000000) & (F.col("sum_" + category) < 45000000), "2")
                                        .otherwise("1"))
                                  .when(F.col("payer_account_type") == "CURRENT", 
                                        F.when(F.col("sum_" + category) < 15000000, "10")
                                        .when((F.col("sum_" + category) >= 15000000) & (F.col("sum_" + category) < 30000000), "9")
                                        .when((F.col("sum_" + category) >= 30000000) & (F.col("sum_" + category) < 45000000), "8")
                                        .when((F.col("sum_" + category) >= 45000000) & (F.col("sum_" + category) < 60000000), "7")
                                        .when((F.col("sum_" + category) >= 60000000) & (F.col("sum_" + category) < 75000000), "6")
                                        .when((F.col("sum_" + category) >= 75000000) & (F.col("sum_" + category) < 90000000), "5")
                                        .when((F.col("sum_" + category) >= 90000000) & (F.col("sum_" + category) < 105000000), "4")
                                        .when((F.col("sum_" + category) >= 105000000) & (F.col("sum_" + category) < 120000000), "3")
                                        .when((F.col("sum_" + category) >= 120000000) & (F.col("sum_" + category) < 135000000), "2")
                                        .otherwise("1"))
                                  .otherwise("Unknown Type"))
    
    return agg_df

# Compute aggregates for category_level1 and category_level2 
agg_df1 = compute_aggregates(result_df, "category_level1", "payer_amount")
agg_df2 = compute_aggregates(result_df, "category_level2", "payer_amount")

# Join the two aggregated DataFrames 
final_agg_df = agg_df1.join(agg_df2, ["payer_account_number", "payer_account_type"], "outer").fillna(0) 

# Calculate the total transaction sum for each account
total_sum_df = result_df.groupBy("payer_account_number").agg(F.sum("payer_amount").alias("total_transaction_sum_paid_as_payer"))

# Calculate how many times each payer_account_number appears in payee_account_number and the sum of payee_amount
appearance_df = result_df.groupBy("payee_account_number") \
                         .agg(F.count("*").alias("as_a_payee_count"), 
                              F.sum("payee_amount").alias("as_a_payee_sum_recieved"))

# Join with unique account details 
unique_account_details = result_df.select("payer_account_number", "account_holder_name", "account_ifsc", "payer_account_type","payee_account_type","payer_account_type").distinct()
final_df = unique_account_details.join(final_agg_df, ["payer_account_number", "payer_account_type"], "left") \
                                 .join(total_sum_df, ["payer_account_number"], "left") \
                                 .join(appearance_df, unique_account_details.payer_account_number == appearance_df.payee_account_number, "left") \
                                 .drop("payee_account_number") \
                                 .fillna(0)

# Display the result
final_df.show()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 36.0 failed 4 times, most recent failure: Lost task 0.3 in stage 36.0 (TID 1287, yball1r044ca19.yesbank.com, executor 5): java.io.IOException: Cannot run program "/opt/anaconda/anaconda2/bin/python": error=2, No such file or directory
	at java.lang.ProcessBuilder.start(ProcessBuilder.java:1048)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:197)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:122)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:95)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:118)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:111)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:66)
	at org.